## Projeto 1 – Previsão de Preços de Casas
### Introdução
O projeto consiste em prever o preço de casas a partir de atributos como área, número de quartos, localização, etc.

In [1]:
from time import process_time

#Bibliotecas e ferramentas necessarias
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

In [6]:
#Carregando os dados
test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')
#print(train.head())
print(train.shape)

(1460, 81)


In [8]:
#Definindo alvo
y = train['SalePrice']
#Definindo feature
X = train.drop(['Id', 'SalePrice'], axis=1)
X_test = test.drop(['Id'], axis=1)

In [13]:
#Separando colunas categóricas
categoric_cols = X.select_dtypes(include=["object"]).columns
#Coluns numericas
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns

In [12]:
#verificando se as colunas categoricas possuem baixa cardinalidade
for col in categoric_cols:
    print(f"{col}:{X[col].nunique()} categorias")

MSZoning:5 categorias
Street:2 categorias
Alley:2 categorias
LotShape:4 categorias
LandContour:4 categorias
Utilities:2 categorias
LotConfig:5 categorias
LandSlope:3 categorias
Neighborhood:25 categorias
Condition1:9 categorias
Condition2:8 categorias
BldgType:5 categorias
HouseStyle:8 categorias
RoofStyle:6 categorias
RoofMatl:8 categorias
Exterior1st:15 categorias
Exterior2nd:16 categorias
MasVnrType:3 categorias
ExterQual:4 categorias
ExterCond:5 categorias
Foundation:6 categorias
BsmtQual:4 categorias
BsmtCond:4 categorias
BsmtExposure:4 categorias
BsmtFinType1:6 categorias
BsmtFinType2:6 categorias
Heating:6 categorias
HeatingQC:5 categorias
CentralAir:2 categorias
Electrical:5 categorias
KitchenQual:4 categorias
Functional:7 categorias
FireplaceQu:5 categorias
GarageType:6 categorias
GarageFinish:3 categorias
GarageQual:5 categorias
GarageCond:5 categorias
PavedDrive:3 categorias
PoolQC:3 categorias
Fence:4 categorias
MiscFeature:4 categorias
SaleType:9 categorias
SaleCondition:6

In [15]:
# transformador para cols numéricas: substitui valores faltantes pela mediana
numeric_transformer = SimpleImputer(strategy="median")
# transformador para cols categoricas : substitui valores faltantes pelo mais frequente
categoric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown="ignore"))
])

In [16]:
# juntando tudo
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categoric_transformer, categoric_cols)
    ]
)

In [18]:
#Minha pipeline
modelo = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('randomforest', RandomForestRegressor(n_estimators=100, random_state=0)),
])

In [20]:
#cross-validation usando MAE
from sklearn.metrics import mean_squared_error
score = -1*cross_val_score(modelo, X, y, cv=5, scoring='neg_mean_absolute_error')
print(f"MAE por folds: {score}"
      f"\nMAE medio: {score.mean():.4f}")

MAE por folds: [17972.48914384 17444.72544521 17775.83342466 16271.63674658
 19033.2214726 ]
MAE medio: 17699.5812


In [22]:
#Treinando o modelo final e prevendo no test
modelo.fit(X, y)
previsao = modelo.predict(X_test)
saida = pd.DataFrame({"Id":test['Id'], "SalePrice":previsao})
saida.to_csv("previsoes.csv", index=False)
print("Arquivo previsoes.csv criado!")

Arquivo previsoes.csv criado!
